# akbo_crawler

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import json

In [2]:
def akbo_crawler(akbo_ajax_url, id_url) :
    
    df_list = []
    
    for i in range(1,101):
        params = {'pageIndex':str(i) , 'groupCode':'12'}
        akbo_ajax_rqst = requests.get(akbo_ajax_url, params = params)
        akbo_ajax_soup = bs(akbo_ajax_rqst.text, 'html.parser')
    
        akbo_list = json.loads(akbo_ajax_rqst.text)
    
        sample_df_i = pd.DataFrame(akbo_list)
        
        df_list.append(sample_df_i)
    
    tmp_df = pd.concat(df_list)
    
    result_tmp = np.empty([3000,5], dtype=object)
    
    for akbo_num, akbo_id in enumerate(tmp_df['AKBO_ID']) : 
    
        id_rqst = requests.get(id_url.format(akbo_id))
        id_soup = bs(id_rqst.content, 'lxml')
    
        # 첫번째 칸에 제목
        result_tmp[akbo_num,0] = id_soup.findAll('dt')[3].text[:-4]
    
        # 두번째 칸부터 나머지 정보 순차적으로
        for i in range(4,100):
            tmp = [value.text.strip() for value in id_soup.findAll('tr')[i].findAll('td')]
            try : 
                tmp[0] == '피아노 3단 악보'
            except IndexError : pass
            else:
                if tmp[0] == '피아노 3단 악보':
                    break
                elif tmp[0] == '피아노&듀엣 악보':
                    break
                elif tmp[0] == 'Solo&피아노 악보':
                    break
                else :
                    pass
    
        result_tmp[akbo_num,1] = tmp[0]
        result_tmp[akbo_num,2] = tmp[1]
        result_tmp[akbo_num,3] = tmp[2]
        result_tmp[akbo_num,4] = tmp[3]
        
    result_df = pd.DataFrame(result_tmp , columns = ['곡명','파트', '설명', 'Key(조)', '페이지'])
    
    final_df = tmp_df[['AK_NAME','ALBUM_NAME','ARTISTLIST']].merge(result_df, left_on = 'AK_NAME', right_on = '곡명')
    
    final_df.to_csv('akbo.csv', index=False)
    
    print('완료!')

In [ ]:
akbo_crawler('http://www.akbobada.com/ajaxPartgList.html','http://www.akbobada.com/musicDetail.html?musicID={}&p=1')